In [1]:
# VC 분석 리팩터 V2 - 임프린팅 전처리 노트북
# 작성자: 홍승재
# 목적: 임프린팅 분석을 위한 데이터 전처리 및 네트워크 분석

import pandas as pd
import numpy as np
import networkx as nx
import warnings
warnings.filterwarnings('ignore')

# vc_analysis 패키지 import
import sys
sys.path.append('/Users/suengj/Documents/Code/Python/Research/VC/refactor_v2')

from vc_analysis import loader, merger
from vc_analysis.data import filter as data_filter
from vc_analysis.network import construction, centrality
from vc_analysis.config import parameters

print("✅ 라이브러리 및 vc_analysis 패키지 로딩 완료")


✅ 라이브러리 및 vc_analysis 패키지 로딩 완료


In [2]:
# vc_analysis 패키지를 사용한 데이터 로딩
data = loader.load_all_data()

# Round 정합성 필터링: firm 레지스트리 기반
from vc_analysis.data import loader as data_loader

# 1) 기본(strict) 모드: firm에 없는 firmname은 round에서 제거
# 현재 분석에서는 네트워크의 노드부터 필터링했다고 보면 됨
data['round'] = data_loader.filter_round_by_firm_registry(
    round_df=data['round'],
    firm_df=data['firm'],
    mode='strict'
)

# 2) 국가 선택 모드 예시: 미국/캐나다 법인만 유지 (필요 시 주석 해제)
# data['round'] = data_loader.filter_round_by_firm_registry(
#     round_df=data['round'],
#     firm_df=data['firm'],
#     mode='nation_select',
#     nation_codes=['US', 'CA']  # 여러 개 가능
# )

print("✅ 데이터 로딩 완료")
print(f"Round: {len(data['round'])} rows")
print(f"Company: {len(data['company'])} rows") 
print(f"Firm: {len(data['firm'])} rows")
print(f"Fund: {len(data['fund'])} rows")

✅ 데이터 로딩 완료
Round: 390011 rows
Company: 61433 rows
Firm: 15603 rows
Fund: 39335 rows


In [3]:
# vc_analysis 패키지를 사용한 VC 네트워크 생성

# 네트워크 설정
START_YEAR = 1990   # 분석 시작 연도
END_YEAR = 2000 # testing
# END_YEAR = 2022     # 분석 종료 연도
TIME_WINDOW = 5     # 네트워크 구성 기간 (년)(예: 1990년이면, 1986~1990년 네트워크)

# 1. 데이터 필터링 (Angel, Other, Null 제외)
filtered_round = data_filter.filter_by_vc_type(
    data['round'], 
    exclude_types=['Angel', 'Other'], 
    type_column='firmtype2',
    exclude_null=True  # Null 값도 제외
)

# 사용 가능한 연도 범위 확인
available_years = sorted(filtered_round['year'].dropna().unique())
print(f"📊 데이터 정보")
print(f"   - 사용 가능한 연도: {available_years[0]} - {available_years[-1]}")
print(f"   - 선택된 분석 기간: {START_YEAR} - {END_YEAR}")
print(f"   - 네트워크 윈도우: {TIME_WINDOW}년")
print(f"   - 생성할 네트워크 수: {END_YEAR - START_YEAR + 1}개\n")

# 2. 네트워크 생성 (START_YEAR부터 END_YEAR까지)
target_years = list(range(START_YEAR, END_YEAR + 1))
networks = construction.construct_networks_for_years(
    filtered_round, 
    years=target_years, 
    time_window=TIME_WINDOW
)

# 3. 중심성 계산
centrality_df = centrality.compute_centralities_for_networks(networks)

# 결과 출력
print(f"\n✅ 네트워크 생성 완료")
print(f"   - 생성된 연도: {len(networks)}개 ({START_YEAR}-{END_YEAR})")
print(f"   - 평균 노드 수: {sum(len(G.nodes()) for G in networks.values()) / len(networks):.0f}")
print(f"   - 평균 엣지 수: {sum(len(G.edges()) for G in networks.values()) / len(networks):.0f}")
print(f"   - 중심성 데이터: {len(centrality_df)} rows")

# 연도별 네트워크 크기 요약
print(f"\n📈 연도별 네트워크 크기:")
for year in sorted(networks.keys()):
    G = networks[year]
    print(f"   {year}: {len(G.nodes())} nodes, {len(G.edges())} edges")


📊 데이터 정보
   - 사용 가능한 연도: 1970 - 2022
   - 선택된 분석 기간: 1990 - 2000
   - 네트워크 윈도우: 5년
   - 생성할 네트워크 수: 11개



Computing centralities: 100%|██████████| 11/11 [00:00<00:00, 11056.16it/s]



✅ 네트워크 생성 완료
   - 생성된 연도: 11개 (1990-2000)
   - 평균 노드 수: 1203
   - 평균 엣지 수: 12611
   - 중심성 데이터: 13228 rows

📈 연도별 네트워크 크기:
   1990: 1121 nodes, 17364 edges
   1991: 1071 nodes, 15821 edges
   1992: 1005 nodes, 13328 edges
   1993: 967 nodes, 11160 edges
   1994: 915 nodes, 9436 edges
   1995: 866 nodes, 8198 edges
   1996: 998 nodes, 8229 edges
   1997: 1179 nodes, 9307 edges
   1998: 1361 nodes, 11079 edges
   1999: 1637 nodes, 13554 edges
   2000: 2108 nodes, 21246 edges


In [ ]:
"""
================================================================================
Initial Network Partner 특성 계산 (Imprinting Analysis) - Option A
================================================================================

연구 목적:
---------
Focal Firm의 Initial Network Partner 특성을 Podolny의 지위 이론에 기반하여 계산.
각 Firm이 최초로 VC 네트워크를 형성한 시점(t1)부터 3년(t1~t3) 동안 맺은 
파트너의 지위(중심성)를 세 가지 방식으로 측정.

방법론적 접근 (Option A: Full History):
-------------------------------------
1. 전체 데이터로 각 Firm의 진짜 Initial Year 식별
   - 예: Firm A가 1972년에 처음 등장 → initial_year = 1972
   - 1990~1995 기간은 성과 측정 기간일 뿐, initial year 식별에는 전체 이력 사용
   
2. 진짜 Imprinting Period의 Initial Partners 추출
   - Firm A: 1972~1974년 파트너들 (진짜 imprinting period)
   - 이 기간의 파트너 중심성도 해당 시점 네트워크로 계산
   
3. 샘플 선택 (Left-Censoring 통제):
   - 1980년 이후 initial ties를 형성한 Firm만 분석
   - 이유: 1970년대 초기 데이터 품질 및 left-censoring 문제

계산 방법 (Partner-Weighted):
-----------------------------
1. Initial Year 식별: 전체 데이터에서 각 Firm의 최초 네트워크 형성 시점(t1) 파악
2. 샘플 필터링: START_YEAR ~ END_YEAR 코호트 선택
3. 필요한 네트워크 생성: 샘플 Firm들의 imprinting period에 필요한 연도만
4. Initial Partner 추출: t1~t3 기간 동안 맺은 모든 unique 파트너 리스트
5. Partner Centrality 계산: 각 파트너의 t1, t2, t3 시점별 중심성 값
   - 각 시점의 중심성은 5-year moving window (t-5 ~ t-1) 네트워크 기반
6. 세 가지 Status 변수 계산 (Partner-Weighted):
   
   Step 1: 각 파트너의 시간 평균 계산
   - Partner B: (cent_t1 + cent_t2) / 2 = avg_B
   - Partner C: (cent_t1 + cent_t3) / 2 = avg_C
   - Partner D: cent_t2 = avg_D
   
   Step 2: 파트너 평균들로부터 집계
   
   Option 1 (Mean Status - Control/Baseline):
   - 각 파트너 평균의 평균
   - 수식: (avg_B + avg_C + avg_D) / |P|
   - 의미: 초기 파트너들의 평균 지위 (파트너 동등 가중)
   
   Option 2 (Max Status - Benefit Variable):
   - 각 파트너 평균 중 최대값
   - 수식: max(avg_B, avg_C, avg_D)
   - 의미: 최고 지위 파트너의 영향력 (혜택)
   
   Option 3 (Min Status - Penalty Variable):
   - 각 파트너 평균 중 최소값
   - 수식: min(avg_B, avg_C, avg_D)
   - 의미: 최저 지위 파트너의 오염 효과 (페널티)

활용:
-----
- Option 1: 기본 통제 변수
- Option 2 & 3: 함께 회귀 모형에 투입하여 지위 이점과 불이익 효과 동시 검증
"""

from vc_analysis.network import imprinting

# ============================================================================
# Step 1: 전체 데이터로 각 Firm의 진짜 Initial Year 식별
# ============================================================================
print("=" * 80)
print("Step 1: Initial Year 식별 (전체 데이터 사용)")
print("=" * 80)

# 전체 데이터로 initial year 식별 (Angel/Other/Null 제외된 filtered_round 사용)
full_initial_year_df = imprinting.identify_initial_year(
    filtered_round,  # 전체 연도 포함 (1970~2022)
    firm_col='firmname',
    year_col='year'
)

print(f"\n✅ Initial Year 식별 완료 (전체 샘플)")
print(f"   - 총 Firm 수: {len(full_initial_year_df)}")
print(f"   - 연도 범위: {full_initial_year_df['initial_year'].min():.0f} ~ {full_initial_year_df['initial_year'].max():.0f}")

# ============================================================================
# Step 2: 샘플 필터링 (코호트 선택)
# ============================================================================
print("\n" + "=" * 80)
print("Step 2: 샘플 필터링 (코호트 선택)")
print("=" * 80)

# 코호트 기간 설정 (START_YEAR ~ END_YEAR에 initial ties 형성한 firm만)
initial_year_df = full_initial_year_df[
    full_initial_year_df['initial_year'].between(START_YEAR, END_YEAR)
].copy()

print(f"\n✅ 샘플 필터링 완료")
print(f"   - 코호트 기간: {START_YEAR} ~ {END_YEAR}")
print(f"   - 전체 Firm: {len(full_initial_year_df)}")
print(f"   - 분석 Firm: {len(initial_year_df)} ({len(initial_year_df)/len(full_initial_year_df)*100:.1f}%)")
print(f"   - 제외 Firm: {len(full_initial_year_df) - len(initial_year_df)}")
print(f"   - 분석 연도 범위: {initial_year_df['initial_year'].min():.0f} ~ {initial_year_df['initial_year'].max():.0f}")
print(f"\n샘플:")
print(initial_year_df.head())


# ============================================================================
# Step 3: 필요한 연도의 네트워크 생성 (Imprinting Period용)
# ============================================================================
print("\n" + "=" * 80)
print("Step 3: 필요한 연도 네트워크 생성")
print("=" * 80)

IMPRINTING_PERIOD = 3  # t1, t2, t3

# 필요한 연도 파악 (각 firm의 imprinting period)
needed_years = set()
for _, row in initial_year_df.iterrows():
    t1 = int(row['initial_year'])
    for offset in range(IMPRINTING_PERIOD):
        needed_years.add(t1 + offset)

needed_years = sorted(needed_years)

print(f"\n📊 네트워크 생성 정보")
print(f"   - Imprinting Period: {IMPRINTING_PERIOD}년")
print(f"   - 분석 Firm 수: {len(initial_year_df)}")
print(f"   - 필요한 네트워크 연도: {len(needed_years)}개")
print(f"   - 연도 범위: {needed_years[0]} ~ {needed_years[-1]}")
print(f"   - Time Window: {TIME_WINDOW}년 (각 연도별)")

# 필요한 연도의 네트워크 생성
print(f"\n🔄 네트워크 생성 중...")
imprinting_networks = construction.construct_networks_for_years(
    filtered_round,  # 전체 데이터 (Angel/Other/Null 제외)
    years=needed_years,
    time_window=TIME_WINDOW
)

# 중심성 계산
print(f"🔄 중심성 계산 중...")
imprinting_centrality_df = centrality.compute_centralities_for_networks(imprinting_networks)

print(f"\n✅ 네트워크 생성 완료")
print(f"   - 생성된 네트워크: {len(imprinting_networks)}개")
print(f"   - 평균 노드 수: {sum(len(G.nodes()) for G in imprinting_networks.values()) / len(imprinting_networks):.0f}")
print(f"   - 평균 엣지 수: {sum(len(G.edges()) for G in imprinting_networks.values()) / len(imprinting_networks):.0f}")
print(f"   - 중심성 데이터: {len(imprinting_centrality_df)} rows")

# 연도별 네트워크 크기 샘플 (처음 5개, 마지막 5개)
print(f"\n📈 연도별 네트워크 크기 (샘플):")
sample_years = needed_years[:5] + needed_years[-5:]
for year in sorted(set(sample_years)):
    if year in imprinting_networks:
        G = imprinting_networks[year]
        print(f"   {year}: {len(G.nodes())} nodes, {len(G.edges())} edges")
if len(needed_years) > 10:
    print(f"   ... ({len(needed_years) - 10}개 연도 생략)")


# ============================================================================
# Step 4: Initial Partners 추출 (진짜 Imprinting Period)
# ============================================================================
print("\n" + "=" * 80)
print("Step 4: Initial Partners 추출 (진짜 Imprinting Period)")

initial_partners_df = imprinting.extract_initial_partners(
    filtered_round,  # 전체 데이터
    imprinting_networks,  # 필요한 연도의 네트워크
    initial_year_df,  # 1980년 이후 firm만
    imprinting_period=IMPRINTING_PERIOD,
    firm_col='firmname'
)

print(f"\n✅ Initial Partners 추출 완료")
print(f"   - 총 Partnership 수: {len(initial_partners_df)}")
print(f"   - Focal Firm 수: {initial_partners_df['firmname'].nunique()}")
print(f"   - Unique Partner 수: {initial_partners_df['initial_partner'].nunique()}")
print(f"   - 연도 범위: {initial_partners_df['tied_year'].min():.0f} ~ {initial_partners_df['tied_year'].max():.0f}")
print(f"\n샘플 (진짜 imprinting period):")
print(initial_partners_df.head(10))


# ============================================================================
# Step 5: Partner Centrality 병합 (해당 시점의 진짜 중심성)
# ============================================================================
print("\n" + "=" * 80)
print("Step 5: Partner Centrality 병합 (해당 시점)")
print("=" * 80)

initial_ties_with_cent = imprinting.calculate_partner_centrality_by_year(
    initial_partners_df,
    imprinting_centrality_df,  # imprinting period 중심성 사용
    firm_col='firmname'
)

print(f"\n✅ Partner Centrality 병합 완료")
print(f"   - 총 행 수: {len(initial_ties_with_cent)}")
print(f"   - 중심성 변수: {[col for col in initial_ties_with_cent.columns if col.startswith('partner_')]}")
print(f"\n샘플:")
print(initial_ties_with_cent[['firmname', 'initial_partner', 'tied_year', 'initial_year', 
                               'partner_dgr_cent', 'partner_btw_cent']].head())


# ============================================================================
# Step 6: Initial Partner Status 계산 (Option 1, 2, 3)
# ============================================================================
print("\n" + "=" * 80)
print("Step 6: Initial Partner Status 계산 (Mean, Max, Min)")
print("=" * 80)

# 모든 중심성 지표에 대해 계산
initial_ties_df = imprinting.compute_all_initial_partner_status(
    initial_ties_with_cent,
    centrality_measures=None,  # None = 모든 중심성 지표 사용
    firm_col='firmname'
)

print(f"\n✅ Initial Partner Status 계산 완료")
print(f"   - Focal Firm 수: {len(initial_ties_df)}")
print(f"   - 변수 수: {len(initial_ties_df.columns)}")
print(f"\n생성된 Status 변수:")
status_vars = [col for col in initial_ties_df.columns if col.startswith('initial_')]
for var in status_vars:
    print(f"   - {var}")

print(f"\n기술 통계 (Degree Centrality 기준):")
print(initial_ties_df[['n_initial_partners', 'n_partner_years',
                        'initial_dgr_cent_mean', 'initial_dgr_cent_max', 
                        'initial_dgr_cent_min']].describe())

print(f"\n샘플:")
print(initial_ties_df[['firmname', 'initial_year', 'n_initial_partners',
                        'initial_dgr_cent_mean', 'initial_dgr_cent_max', 
                        'initial_dgr_cent_min']].head())

print("\n" + "=" * 80)
print("✅ Initial Network Partner 특성 계산 완료! (Option A)")
print("=" * 80)
print(f"\n최종 데이터셋: initial_ties_df")
print(f"   - Shape: {initial_ties_df.shape}")
print(f"   - Focal Firm 수: {len(initial_ties_df)}")
if 'initial_year' in initial_ties_df.columns and len(initial_ties_df) > 0:
    min_year = initial_ties_df['initial_year'].min()
    max_year = initial_ties_df['initial_year'].max()
    print(f"   - Initial Year 범위: {min_year} ~ {max_year}")
else:
    print(f"   - Initial Year 정보 없음")
print(f"   - 샘플 기준: {START_YEAR}~{END_YEAR} 코호트")
print(f"\n방법론:")
print(f"   - Option A (Full History): 전체 데이터로 진짜 initial year 식별")
print(f"   - 진짜 imprinting period (t1~t3)의 파트너 특성 계산")
print(f"   - 해당 시점의 정확한 네트워크 중심성 사용")
print(f"   - Partner-weighted: 각 파트너의 시간 평균을 먼저 계산 후 집계")
print(f"\n활용:")
print(f"   - Option 1 (Mean): 기본 통제 변수 (파트너 평균의 평균)")
print(f"   - Option 2 (Max) & Option 3 (Min): 지위 이점과 불이익 효과 검증")
print(f"\n유연성:")
print(f"   - 성과 데이터와 merge하여 분석")
print(f"   - 또는 initial_year로 다른 코호트 분석 가능")
print(f"   - 예: initial_ties_df[initial_ties_df['initial_year'].between(1990, 1995)]")


Step 1: Initial Year 식별 (전체 데이터 사용)

✅ Initial Year 식별 완료 (전체 샘플)
   - 총 Firm 수: 11623
   - 연도 범위: 1970 ~ 2022

Step 2: 샘플 필터링 (코호트 선택)

✅ 샘플 필터링 완료
   - 코호트 기간: 1990 ~ 2000
   - 전체 Firm: 11623
   - 분석 Firm: 2069 (17.8%)
   - 제외 Firm: 9554
   - 분석 연도 범위: 1990 ~ 2000

샘플:
                   firmname  initial_year
3          100 X Better Inc          2000
21  180 Degree Capital Corp          1996
29  1ST GLOBAL VENTURES INC          2000
32         1To1 Ventures LP          2000
42             212 Ventures          2000

Step 3: 필요한 연도 네트워크 생성

📊 네트워크 생성 정보
   - Imprinting Period: 3년
   - 분석 Firm 수: 2069
   - 필요한 네트워크 연도: 13개
   - 연도 범위: 1990 ~ 2002
   - Time Window: 5년 (각 연도별)

🔄 네트워크 생성 중...


Constructing networks: 100%|██████████| 13/13 [00:00<00:00, 13888.42it/s]


🔄 중심성 계산 중...


Computing centralities: 100%|██████████| 13/13 [00:00<00:00, 6687.02it/s]


In [ ]:
"""
================================================================================
Firm-Level 기본 변수 생성 (Firm-Year 기준)
================================================================================

생성 변수:
----------
1. firmage: 펌 연령 (year - founding_year)
2. industry_blau: 투자 다양성 (Blau index by company industry)
3. perf_IPO: IPO 성과 (sum of ipoExit)
4. perf_MnA: M&A 성과 (sum of MnAExit)
5. perf_all: 전체 성과 (perf_IPO + perf_MnA)
6. early_stage_ratio: Early stage 투자 비율 (Seed, Series A, B)
7. firm_hq_CAMA: 본사 위치 더미 (CA/MA = 1, else = 0)
8. inv_amt: 연간 총 투자 금액 (thousands)
9. inv_num: 연간 총 투자 건수

데이터 구조:
-----------
- Level: Firm-Year
- Key: (firmname, year)
- 분석 기간: START_YEAR ~ END_YEAR
"""

from vc_analysis.variables import firm_variables

print("=" * 80)
print("Firm-Level 기본 변수 생성")
print("=" * 80)

# 기본 변수 생성 (firm-year 기준)
print(f"\n🔄 변수 생성 중...")
print(f"   - 분석 기간: {START_YEAR} ~ {END_YEAR}")
print(f"   - 생성 변수: 9개")

firm_vars_df = firm_variables.calculate_all_firm_variables(
    round_df=data['round'],
    company_df=data['company'],
    firm_df=data['firm'],
    year_col='year'
)

# VC Reputation 계산
reputation_df = firm_variables.calculate_vc_reputation(
    round_df=data['round'],
    company_df=data['company'],
    fund_df=data['fund'],
    year_col='year',
    window_years=5
)

# Market Heat 계산 (industry-level)
market_heat_df = firm_variables.calculate_market_heat(
    fund_df=data['fund'],
    year_col='year',
    fundyear_col='fundyear',
    fundname_col='fundname'
)

# New Venture Funding Demand 계산 (industry-level, current year)
# 먼저 필요한 컬럼 확인
print("🔍 New Venture Funding Demand 계산 전 컬럼 확인:")
print(f"  - round_df 컬럼: RoundNumber 존재 여부 = {'RoundNumber' in data['round'].columns}")
if 'RoundNumber' in data['round'].columns:
    print(f"    RoundNumber non-null: {data['round']['RoundNumber'].notna().sum():,} / {len(data['round']):,}")
    print(f"    RoundNumber unique values 샘플: {data['round']['RoundNumber'].dropna().unique()[:10]}")
print(f"  - company_df 컬럼: comnation 존재 여부 = {'comnation' in data['company'].columns}")
if 'comnation' in data['company'].columns:
    print(f"    comnation unique values 샘플: {data['company']['comnation'].dropna().unique()[:10]}")
    print(f"    US companies: {(data['company']['comnation'] == 'United States').sum():,}")

new_venture_demand_df = firm_variables.calculate_new_venture_funding_demand(
    round_df=data['round'],
    company_df=data['company'],
    year_col='year',
    roundnumber_col='RoundNumber',
    comname_col='comname',
    comnation_col='comnation',
    us_nation='US'  # 미국은 'US'로 표기됨 (company 데이터에서)
)

# 결과 확인
print(f"\n📊 New Venture Funding Demand 계산 결과:")
print(f"  - 생성된 데이터 행 수: {len(new_venture_demand_df)}")
if len(new_venture_demand_df) > 0:
    print(f"  - 연도 범위: {new_venture_demand_df['year'].min()} - {new_venture_demand_df['year'].max()}")
    print(f"  - new_venture_demand 샘플:")
    print(new_venture_demand_df.head(10))
else:
    print(f"  ⚠️  데이터가 비어있습니다!")

# Zero-fill perf_* only (post-merge)
from vc_analysis.variables.firm_variables import fill_missing_performance_with_zero

firm_vars_df = fill_missing_performance_with_zero(
    firm_vars_df,
    columns=['perf_IPO', 'perf_MnA', 'perf_all'],
    inplace=False
)

print(f"\n✅ 변수 생성 완료")
print(f"   - 총 행 수: {len(firm_vars_df):,}")
print(f"   - Firm 수: {firm_vars_df['firmname'].nunique():,}")
print(f"   - 연도 범위: {firm_vars_df['year'].min():.0f} ~ {firm_vars_df['year'].max():.0f}")
print(f"   - 생성 변수: {len(firm_vars_df.columns) - 2}개 (firmname, year 제외)")

# 분석 기간으로 필터링
firm_vars_df_filtered = firm_vars_df[
    firm_vars_df['year'].between(START_YEAR, END_YEAR)
].copy()

print(f"\n📊 분석 기간 필터링 ({START_YEAR}~{END_YEAR})")
print(f"   - 필터링 전: {len(firm_vars_df):,} rows")
print(f"   - 필터링 후: {len(firm_vars_df_filtered):,} rows")
print(f"   - Firm 수: {firm_vars_df_filtered['firmname'].nunique():,}")

# 기술 통계
print(f"\n📈 기술 통계:")
desc_cols = ['firmage', 'industry_blau', 'perf_all', 'early_stage_ratio', 
             'inv_amt', 'inv_num']
print(firm_vars_df_filtered[desc_cols].describe())

print(f"\n샘플 (상위 5개):")
print(firm_vars_df_filtered.head())

print(f"\n✅ Firm-Level 기본 변수 생성 완료!")
print(f"   - 데이터셋: firm_vars_df_filtered")
print(f"   - Shape: {firm_vars_df_filtered.shape}")

# Geographic Distance 계산 (VC-Company, VC-Co-Partner)
from vc_analysis.distance import geographic

print("\n" + "=" * 80)
print("Geographic Distance 계산 (ZIP 코드 기반)")
print("=" * 80)

print("\n🔄 Geographic Distance 계산 중...")
print("   - VC-Company 거리 (firm-year level)")
print("   - VC-Co-Partner 거리 (firm-year level)")

# ZIP code database 구축 (한 번만)
zipcode_db = geographic.build_zipcode_database(
    firm_df=data['firm'],
    company_df=data['company'],
    firmzip_col='firmzip',
    comzip_col='comzip'
)

# 1) VC-Company 거리 계산
geo_dist_company_df = geographic.calculate_vc_company_distances(
    round_df=data['round'],
    firm_df=data['firm'],
    company_df=data['company'],
    zipcode_db=zipcode_db,
    firm_col='firmname',
    comname_col='comname',
    year_col='year',
    firmzip_col='firmzip',
    comzip_col='comzip',
    amount_col='RoundAmountDisclosedThou'  # 가중 평균용
)

# 2) VC-Co-Partner 거리 계산
geo_dist_copartner_df = geographic.calculate_vc_copartner_distances(
    round_df=data['round'],
    firm_df=data['firm'],
    zipcode_db=zipcode_db,
    firm_col='firmname',
    comname_col='comname',
    year_col='year',
    firmzip_col='firmzip',
    amount_col='RoundAmountDisclosedThou'  # 가중 평균용
)

print(f"\n✅ Geographic Distance 계산 완료")
print(f"   - VC-Company: {len(geo_dist_company_df)} firm-year observations")
print(f"   - VC-Co-Partner: {len(geo_dist_copartner_df)} firm-year observations")



Firm-Level 기본 변수 생성

🔄 변수 생성 중...
   - 분석 기간: 1990 ~ 2000
   - 생성 변수: 9개
🔍 New Venture Funding Demand 계산 전 컬럼 확인:
  - round_df 컬럼: RoundNumber 존재 여부 = True
    RoundNumber non-null: 390,011 / 390,011
    RoundNumber unique values 샘플: [ 2  1  3  6  7  4  8  5  9 10]
  - company_df 컬럼: comnation 존재 여부 = True
    comnation unique values 샘플: ['US']
Categories (1, object): ['US']
    US companies: 0

📊 New Venture Funding Demand 계산 결과:
  - 생성된 데이터 행 수: 53
  - 연도 범위: 1970 - 2022
  - new_venture_demand 샘플:
   year  new_venture_demand
0  1970            2.397895
1  1971            2.995732
2  1972            3.332205
3  1973            3.555348
4  1974            3.258097
5  1975            3.218876
6  1976            3.583519
7  1977            3.871201
8  1978            4.430817
9  1979            4.779123

✅ 변수 생성 완료
   - 총 행 수: 68,045
   - Firm 수: 12,069
   - 연도 범위: 1970 ~ 2022
   - 생성 변수: 12개 (firmname, year 제외)

📊 분석 기간 필터링 (1990~2000)
   - 필터링 전: 68,045 rows
   - 필터링 후: 9,813 rows
   -

In [ ]:
"""
================================================================================
최종 데이터셋 생성: Firm-Year 기준 Merge
================================================================================

Merge 전략:
----------
1. 기준 데이터: centrality_df (firm-year 중심성 데이터)
   - Key: (firmname, year)
   - 분석 기간: START_YEAR ~ END_YEAR
   
2. Merge 데이터:
   a) firm_vars_df_filtered: Firm-level 기본 변수 (firm-year)
   b) initial_ties_df: Initial partner 특성 (firm-level, initial_year로 매칭)
   c) reputation_df: VC Reputation 변수 (firm-year)

3. Merge 방법:
   - Left join: firm_vars_df_filtered를 기준으로
   - firm_vars: (firmname, year) 매칭
   - initial_ties: firmname 매칭 (initial_year는 정보 변수로 유지)
   - reputation: (firmname, year) 매칭

최종 데이터 구조:
---------------
- Level: Firm-Year
- Key: (firmname, year)
- 변수 그룹:
  * Network centrality (8개): dgr_cent, btw_cent, pwr_*, constraint, ego_dens
  * Firm basics (9개): firmage, industry_blau, perf_*, early_stage_ratio, firm_hq, inv_amt, inv_num
  * Initial partner status (24개): initial_*_mean/max/min for each centrality
  * VC Reputation (7개): rep_portfolio_count, rep_total_invested, rep_avg_fum, rep_funds_raised, rep_ipos, fundingAge, VC_reputation
  * Meta info: initial_year, n_initial_partners, n_partner_years
"""

print("=" * 80)
print("최종 데이터셋 생성: Firm-Year 기준 Merge")
print("=" * 80)

# ============================================================================
# Step 1: 기준 데이터 준비 (Firm Variables - R 코드 로직)
# ============================================================================
print("\n📊 Step 1: 기준 데이터 준비")

print(f"   - 기준: firm_vars_df_filtered (투자 집행 firm-year)")
print(f"   - 기간: {START_YEAR} ~ {END_YEAR}")
print(f"   - Shape: {firm_vars_df_filtered.shape}")
print(f"   - Firm 수: {firm_vars_df_filtered['firmname'].nunique():,}")
print(f"   - 기본 변수: {[col for col in firm_vars_df_filtered.columns if col not in ['firmname', 'year']]}")

# ============================================================================
# Step 2: Network Centrality Merge (R 코드: left_join(dta, cent_1y))
# ============================================================================
print("\n📊 Step 2: Network Centrality Merge")

# Centrality 데이터를 분석 기간으로 필터링
centrality_df_filtered = centrality_df[
    centrality_df['year'].between(START_YEAR, END_YEAR)
].copy()

# R 코드 로직: firm_vars를 기준으로 centrality를 left join
final_df = firm_vars_df_filtered.merge(
    centrality_df_filtered,
    on=['firmname', 'year'],
    how='left'
)

print(f"   - Merge 방법: Left join on (firmname, year)")
print(f"   - 기준 rows: {len(firm_vars_df_filtered):,} (투자 집행 firm-year)")
print(f"   - Centrality rows: {len(centrality_df_filtered):,} (네트워크 등장 firm-year)")
print(f"   - 최종 Shape: {final_df.shape}")
print(f"   - 추가된 변수: {len(centrality_df_filtered.columns) - 2}개 (중심성)")

# Missing 체크 (Centrality 변수)
centrality_cols = [col for col in centrality_df_filtered.columns if col not in ['firmname', 'year']]
missing_counts = final_df[centrality_cols].isnull().sum()
if missing_counts.sum() > 0:
    print(f"\n   ⚠️  Centrality Missing 발견 (네트워크 미등장):")
    for col, count in missing_counts[missing_counts > 0].items():
        print(f"      - {col}: {count:,} ({count/len(final_df)*100:.1f}%)")
else:
    print(f"   ✅ Centrality Missing 없음 (모든 firm이 네트워크에 등장)")

# ============================================================================
# Step 3: Initial Period Variables 계산 (t1~t3 기간 변수 집계)
# ============================================================================
print("\n📊 Step 3: Initial Period Variables 계산 (t1~t3)")

from vc_analysis.network import imprinting

# Initial period 변수 계산 (t1~t3 기간 동안의 투자 행위/특성 집계)
initial_period_vars_df = imprinting.calculate_initial_period_variables(
    initial_year_df=initial_year_df,
    firm_vars_df=firm_vars_df_filtered,  # 분석 기간 필터링된 firm-year 변수
    market_heat_df=market_heat_df_filtered if 'market_heat_df_filtered' in locals() else None,
    new_venture_demand_df=new_venture_demand_df_filtered if 'new_venture_demand_df_filtered' in locals() else None,
    imprinting_period=3,  # t1~t3 (3년)
    firm_col='firmname',
    year_col='year'
)

print(f"   - 생성된 변수: {len([c for c in initial_period_vars_df.columns if c.startswith('initial_')])}개")
print(f"   - 변수 목록:")
for col in initial_period_vars_df.columns:
    if col.startswith('initial_'):
        non_null = initial_period_vars_df[col].notna().sum()
        print(f"      - {col}: {non_null:,} non-null ({non_null/len(initial_period_vars_df)*100:.1f}%)")

# Initial period geographic distance 계산 (t1~t3 기간 동안의 공동 투자 파트너 거리)
print("\n🔄 Initial Period Geographic Distance 계산 중...")
initial_geo_dist_df = imprinting.calculate_initial_period_geographic_distances(
    initial_year_df=initial_year_df,
    copartner_dist_df=geo_dist_copartner_df,  # 전체 기간 co-partner 거리 데이터
    imprinting_period=3,  # t1~t3 (3년)
    firm_col='firmname',
    year_col='year'
)

print(f"   - 생성된 변수: {len([c for c in initial_geo_dist_df.columns if c.startswith('initial_geo_dist_copartner')])}개")

# ============================================================================
# Step 4: Initial Partner Status Merge (R 코드: left_join(dta, initial_partner))
# ============================================================================
print("\n📊 Step 4: Initial Partner Status Merge")

# initial_ties_df에서 중복 제거 (firmname 기준)
# initial_year는 각 firm마다 하나만 존재해야 함
if initial_ties_df['firmname'].duplicated().any():
    print(f"   ⚠️  중복 firmname 발견: {initial_ties_df['firmname'].duplicated().sum()}개")
    print(f"   → 첫 번째 행만 유지")
    initial_ties_df_unique = initial_ties_df.drop_duplicates(subset=['firmname'], keep='first')
else:
    initial_ties_df_unique = initial_ties_df

final_df = final_df.merge(
    initial_ties_df_unique,
    on='firmname',
    how='left'
)

print(f"   - Merge 방법: Left join on firmname")
print(f"   - Shape: {final_df.shape}")
print(f"   - 추가된 변수: {len(initial_ties_df_unique.columns) - 1}개")

# ============================================================================
# Step 4-1: Initial Period Variables Merge (t1~t3 기간 변수)
# ============================================================================
print("\n📊 Step 4-1: Initial Period Variables Merge (t1~t3)")

final_df = final_df.merge(
    initial_period_vars_df,
    on='firmname',
    how='left'
)

print(f"   - Merge 방법: Left join on firmname")
print(f"   - Shape: {final_df.shape}")
print(f"   - 추가된 변수: {len([c for c in initial_period_vars_df.columns if c.startswith('initial_')])}개")
print(f"   - 변수 목록:")
for col in initial_period_vars_df.columns:
    if col.startswith('initial_') and col != 'initial_year':
        non_null = final_df[col].notna().sum()
        print(f"      - {col}: {non_null:,} non-null ({non_null/len(final_df)*100:.1f}%)")

# ============================================================================
# Step 4-2: Initial Period Geographic Distance Merge (t1~t3 기간 공동 투자 파트너 거리)
# ============================================================================
print("\n📊 Step 4-2: Initial Period Geographic Distance Merge (t1~t3)")

final_df = final_df.merge(
    initial_geo_dist_df,
    on='firmname',
    how='left'
)

print(f"   - Merge 방법: Left join on firmname")
print(f"   - Shape: {final_df.shape}")
print(f"   - 추가된 변수: {len([c for c in initial_geo_dist_df.columns if c.startswith('initial_geo_dist_copartner')])}개")
print(f"   - 변수 목록:")
for col in initial_geo_dist_df.columns:
    if col.startswith('initial_geo_dist_copartner'):
        non_null = final_df[col].notna().sum()
        print(f"      - {col}: {non_null:,} non-null ({non_null/len(final_df)*100:.1f}%)")

# Initial ties가 있는 firm 비율
has_initial_ties = final_df['initial_year'].notna()
matched_count = int(has_initial_ties.sum())
unmatched_count = int((~has_initial_ties).sum())
print(f"\n   📈 Initial Ties 매칭 현황:")
print(f"      - 매칭된 firm-year: {matched_count:,} ({matched_count/len(final_df)*100:.1f}%)")
print(f"      - 매칭 안 된 firm-year: {unmatched_count:,} ({unmatched_count/len(final_df)*100:.1f}%)")
print(f"      - Unique firm (매칭): {final_df[has_initial_ties]['firmname'].nunique():,}")
print(f"      - Unique firm (전체): {final_df['firmname'].nunique():,}")

# ============================================================================
# Step 5: Geographic Distance Merge (VC-Company, VC-Co-Partner)
# ============================================================================
print("\n📊 Step 5: Geographic Distance Merge")

# Geographic distance 데이터를 분석 기간으로 필터링
geo_dist_company_df_filtered = geo_dist_company_df[
    geo_dist_company_df['year'].between(START_YEAR, END_YEAR)
].copy()

geo_dist_copartner_df_filtered = geo_dist_copartner_df[
    geo_dist_copartner_df['year'].between(START_YEAR, END_YEAR)
].copy()

# Merge VC-Company distances
final_df = final_df.merge(
    geo_dist_company_df_filtered,
    on=['firmname', 'year'],
    how='left'
)

print(f"   - VC-Company 거리 Merge: Left join on (firmname, year)")
print(f"   - Shape: {final_df.shape}")
print(f"   - 추가된 변수: {len([c for c in geo_dist_company_df_filtered.columns if c.startswith('geo_dist_company')])}개")

# Merge VC-Co-Partner distances
final_df = final_df.merge(
    geo_dist_copartner_df_filtered,
    on=['firmname', 'year'],
    how='left'
)

print(f"   - VC-Co-Partner 거리 Merge: Left join on (firmname, year)")
print(f"   - Shape: {final_df.shape}")
print(f"   - 추가된 변수: {len([c for c in geo_dist_copartner_df_filtered.columns if c.startswith('geo_dist_copartner')])}개")

# Missing 체크
geo_company_cols = [c for c in final_df.columns if c.startswith('geo_dist_company')]
geo_copartner_cols = [c for c in final_df.columns if c.startswith('geo_dist_copartner')]

if geo_company_cols:
    missing_company = final_df[geo_company_cols].isnull().sum()
    if missing_company.sum() > 0:
        print(f"\n   ⚠️  VC-Company Distance Missing:")
        for col, count in missing_company[missing_company > 0].items():
            print(f"      - {col}: {count:,} ({count/len(final_df)*100:.1f}%)")

if geo_copartner_cols:
    missing_copartner = final_df[geo_copartner_cols].isnull().sum()
    if missing_copartner.sum() > 0:
        print(f"\n   ⚠️  VC-Co-Partner Distance Missing:")
        for col, count in missing_copartner[missing_copartner > 0].items():
            print(f"      - {col}: {count:,} ({count/len(final_df)*100:.1f}%)")

# ============================================================================
# Step 6: VC Reputation Merge
# ============================================================================
print("\n📊 Step 6: VC Reputation Merge")

# Reputation 데이터를 분석 기간으로 필터링
reputation_df_filtered = reputation_df[
    reputation_df['year'].between(START_YEAR, END_YEAR)
].copy()

# Merge reputation data (firm-year level)
final_df = final_df.merge(
    reputation_df_filtered,
    on=['firmname', 'year'],
    how='left'
)

print(f"   - Merge 방법: Left join on (firmname, year)")
print(f"   - Shape: {final_df.shape}")
print(f"   - 추가된 변수: {len(reputation_df_filtered.columns) - 2}개 (VC Reputation)")

# Reputation 변수 확인
rep_vars = [col for col in final_df.columns if col.startswith('rep_') or col == 'VC_reputation' or col == 'fundingAge']
if rep_vars:
    print(f"   - Reputation 변수: {rep_vars}")
    
    # Missing 체크
    rep_missing = final_df[rep_vars].isnull().sum()
    if rep_missing.sum() > 0:
        print(f"\n   ⚠️  Reputation Missing 발견:")
        for col, count in rep_missing[rep_missing > 0].items():
            print(f"      - {col}: {count:,} ({count/len(final_df)*100:.1f}%)")
    else:
        print(f"   ✅ Reputation Missing 없음")
else:
    print(f"   ⚠️  Reputation 변수를 찾을 수 없습니다")

# ============================================================================
# Step 7: Market Heat Merge (Industry-Level, Current Year)
# ============================================================================
print("\n📊 Step 7: Market Heat Merge (Industry-Level, Current Year)")

# Market Heat 데이터를 분석 기간으로 필터링
market_heat_df_filtered = market_heat_df[
    market_heat_df['year'].between(START_YEAR, END_YEAR)
].copy()

# Merge market heat data (industry-level: 같은 연도면 모든 firm에 동일한 값)
final_df = final_df.merge(
    market_heat_df_filtered,
    on='year',
    how='left'
)

print(f"   - Merge 방법: Left join on year (industry-level)")
print(f"   - Shape: {final_df.shape}")
print(f"   - 추가된 변수: 1개 (market_heat)")

# Market Heat 변수 확인
if 'market_heat' in final_df.columns:
    # Missing 체크
    market_heat_missing = final_df['market_heat'].isnull().sum()
    if market_heat_missing > 0:
        print(f"\n   ⚠️  Market Heat Missing 발견:")
        print(f"      - market_heat: {market_heat_missing:,} ({market_heat_missing/len(final_df)*100:.1f}%)")
    else:
        print(f"   ✅ Market Heat Missing 없음")
    
    # 기술 통계
    print(f"\n   📈 Market Heat 기술 통계:")
    print(f"      - 범위: {final_df['market_heat'].min():.3f} ~ {final_df['market_heat'].max():.3f}")
    print(f"      - 평균: {final_df['market_heat'].mean():.3f}")
    hot_markets = (final_df['market_heat'] > 0).sum()
    cold_markets = (final_df['market_heat'] < 0).sum()
    print(f"      - Hot markets (>0): {hot_markets:,} ({hot_markets/len(final_df)*100:.1f}%)")
    print(f"      - Cold markets (<0): {cold_markets:,} ({cold_markets/len(final_df)*100:.1f}%)")
else:
    print(f"   ⚠️  Market Heat 변수를 찾을 수 없습니다")

# ============================================================================
# Step 8: New Venture Funding Demand Merge (Industry-Level, Current Year)
# ============================================================================
print("\n📊 Step 8: New Venture Funding Demand Merge (Industry-Level, Current Year)")

# New Venture Funding Demand 데이터를 분석 기간으로 필터링
new_venture_demand_df_filtered = new_venture_demand_df[
    new_venture_demand_df['year'].between(START_YEAR, END_YEAR)
].copy()

# Merge new venture demand data (industry-level: 같은 연도면 모든 firm에 동일한 값)
final_df = final_df.merge(
    new_venture_demand_df_filtered,
    on='year',
    how='left'
)

print(f"   - Merge 방법: Left join on year (industry-level)")
print(f"   - Shape: {final_df.shape}")
print(f"   - 추가된 변수: 1개 (new_venture_demand, current year)")
print(f"   - 주의: Raw 데이터셋이므로 당해 연도 기준. Panel 분석 시 lagging 필요")

# New Venture Funding Demand 변수 확인
if 'new_venture_demand' in final_df.columns:
    # Missing 체크
    demand_missing = final_df['new_venture_demand'].isnull().sum()
    if demand_missing > 0:
        print(f"\n   ⚠️  New Venture Funding Demand Missing 발견:")
        print(f"      - new_venture_demand: {demand_missing:,} ({demand_missing/len(final_df)*100:.1f}%)")
    else:
        print(f"   ✅ New Venture Funding Demand Missing 없음")
    
    # 기술 통계
    print(f"\n   📈 New Venture Funding Demand 기술 통계:")
    print(f"      - 범위: {final_df['new_venture_demand'].min():.3f} ~ {final_df['new_venture_demand'].max():.3f}")
    print(f"      - 평균: {final_df['new_venture_demand'].mean():.3f}")
    print(f"      - 표준편차: {final_df['new_venture_demand'].std():.3f}")
else:
    print(f"   ⚠️  New Venture Funding Demand 변수를 찾을 수 없습니다")

# ============================================================================
# Step 7: 최종 데이터셋 정보
# ============================================================================
print("\n" + "=" * 80)
print("✅ 최종 데이터셋 생성 완료!")
print("=" * 80)

print(f"\n📊 최종 데이터셋: final_df")
print(f"   - Shape: {final_df.shape}")
print(f"   - Level: Firm-Year")
print(f"   - Key: (firmname, year)")
print(f"   - 기간: {final_df['year'].min():.0f} ~ {final_df['year'].max():.0f}")
print(f"   - Firm 수: {final_df['firmname'].nunique():,}")
print(f"   - 총 변수: {len(final_df.columns)}개 (Market Heat, New Venture Funding Demand 포함)")

# 변수 그룹별 정리
print(f"\n📋 변수 그룹:")

# 1. Key 변수
key_vars = ['firmname', 'year']
print(f"\n   1. Key 변수 ({len(key_vars)}개):")
for var in key_vars:
    print(f"      - {var}")

# 2. Network centrality
network_vars = [col for col in final_df.columns 
                if col in ['dgr_cent', 'btw_cent', 'pwr_max', 'pwr_p0', 'pwr_p75', 
                          'pwr_p99', 'constraint', 'ego_dens']]
print(f"\n   2. Network Centrality ({len(network_vars)}개):")
for var in network_vars:
    print(f"      - {var}")

# 3. Firm basics
firm_basic_vars = [col for col in final_df.columns 
                   if col in ['firmage', 'industry_blau', 'perf_IPO', 'perf_MnA', 
                             'perf_all', 'early_stage_ratio', 'firm_hq_CAMA', 
                             'inv_amt', 'inv_num']]
print(f"\n   3. Firm Basics ({len(firm_basic_vars)}개):")
for var in firm_basic_vars:
    print(f"      - {var}")

# 4. Initial partner status
initial_status_vars = [col for col in final_df.columns if col.startswith('initial_') 
                       and col.endswith(('_mean', '_max', '_min'))]
print(f"\n   4. Initial Partner Status ({len(initial_status_vars)}개):")
print(f"      - {len(initial_status_vars)//3}개 중심성 × 3개 집계 (mean/max/min)")
print(f"      - 예: {initial_status_vars[:3]}")

# 5. VC Reputation
rep_vars = [col for col in final_df.columns 
            if col.startswith('rep_') or col == 'VC_reputation' or col == 'fundingAge']
print(f"\n   5. VC Reputation ({len(rep_vars)}개):")
if rep_vars:
    for var in rep_vars:
        print(f"      - {var}")
else:
    print(f"      - (변수 없음)")

# 6. Geographic Distance (Firm-Year Level)
geo_company_vars = [col for col in final_df.columns if col.startswith('geo_dist_company')]
geo_copartner_vars = [col for col in final_df.columns if col.startswith('geo_dist_copartner')]
print(f"\n   6. Geographic Distance - VC-Company ({len(geo_company_vars)}개):")
if geo_company_vars:
    for var in geo_company_vars:
        print(f"      - {var}")
else:
    print(f"      - (변수 없음)")

print(f"\n   7. Geographic Distance - VC-Co-Partner ({len(geo_copartner_vars)}개):")
if geo_copartner_vars:
    for var in geo_copartner_vars:
        print(f"      - {var}")
else:
    print(f"      - (변수 없음)")

# 7. Market Heat (Industry-Level)
market_heat_vars = [col for col in final_df.columns if col == 'market_heat']
print(f"\n   8. Market Heat ({len(market_heat_vars)}개):")
if market_heat_vars:
    for var in market_heat_vars:
        print(f"      - {var}")
else:
    print(f"      - (변수 없음)")

# 8. New Venture Funding Demand (Industry-Level, Current Year)
demand_vars = [col for col in final_df.columns if col == 'new_venture_demand']
print(f"\n   9. New Venture Funding Demand ({len(demand_vars)}개):")
if demand_vars:
    for var in demand_vars:
        print(f"      - {var} (current year, ln(count of first-round US ventures in year t))")
        print(f"        → Panel 분석 시 lagging 필요 (예: year t-1 사용)")
else:
    print(f"      - (변수 없음)")

# 9. Meta info
meta_vars = [col for col in final_df.columns 
             if col in ['initial_year', 'n_initial_partners', 'n_partner_years']]
print(f"\n   10. Meta Info ({len(meta_vars)}개):")
for var in meta_vars:
    print(f"      - {var}")

# 기술 통계 샘플
print(f"\n📈 기술 통계 (주요 변수):")
sample_vars = ['dgr_cent', 'firmage', 'industry_blau', 'perf_all', 
               'inv_num', 'initial_dgr_cent_mean']
print(final_df[sample_vars].describe())

print(f"\n샘플 (상위 5개):")
display_cols = ['firmname', 'year', 'dgr_cent', 'firmage', 'perf_all', 
                'initial_dgr_cent_mean', 'initial_year']
print(final_df[display_cols].head())

print(f"\n💾 데이터 저장 준비 완료!")
print(f"   - 변수명: final_df")
print(f"   - 용도: 회귀 분석, 패널 데이터 분석")
print(f"   - 추천 저장 형식: Parquet (빠른 I/O)")


# ============================================================================
# Step 9: 최종 Missing data 확인
# ============================================================================
# Firm basics에서만 결측 확인: firmage, firm_hq (centrality 결측은 무시)
check_cols = [c for c in ['firmage', 'firm_hq'] if c in final_df.columns]
missing_mask = final_df[check_cols].isna().any(axis=1)

# 결과 요약
print("🧭 Firm basics 결측 요약 (centrality 제외):")
for c in check_cols:
    cnt = int(final_df[c].isna().sum())
    print(f"  - {c} missing: {cnt:,} ({cnt/len(final_df)*100:.1f}%)")

# 결측 행만 추출
missing_rows = final_df.loc[missing_mask, ['firmname', 'year'] + check_cols] \
                       .sort_values(['firmname', 'year']) \
                       .reset_index(drop=True)

print(f"\n🔎 결측 행 수: {len(missing_rows):,} / {len(final_df):,}")
missing_rows.head(30)

최종 데이터셋 생성: Firm-Year 기준 Merge

📊 Step 1: 기준 데이터 준비
   - 기준: firm_vars_df_filtered (투자 집행 firm-year)
   - 기간: 1990 ~ 2000
   - Shape: (9813, 14)
   - Firm 수: 2,999
   - 기본 변수: ['firmage', 'industry_blau', 'perf_IPO', 'perf_MnA', 'perf_all', 'early_stage_ratio', 'inv_amt', 'inv_num', 'firm_hq_CA', 'firm_hq_MA', 'firm_hq_NY', 'firm_hq']

📊 Step 2: Network Centrality Merge
   - Merge 방법: Left join on (firmname, year)
   - 기준 rows: 9,813 (투자 집행 firm-year)
   - Centrality rows: 13,228 (네트워크 등장 firm-year)
   - 최종 Shape: (9813, 22)
   - 추가된 변수: 8개 (중심성)

   ⚠️  Centrality Missing 발견 (네트워크 미등장):
      - dgr_cent: 2,579 (26.3%)
      - btw_cent: 2,579 (26.3%)
      - pwr_max: 2,579 (26.3%)
      - pwr_p0: 2,579 (26.3%)
      - pwr_p75: 2,579 (26.3%)
      - pwr_p99: 2,579 (26.3%)
      - constraint: 2,579 (26.3%)
      - ego_dens: 2,579 (26.3%)

📊 Step 3: Initial Period Variables 계산 (t1~t3)
   - 생성된 변수: 8개
   - 변수 목록:
      - initial_year: 2,069 non-null (100.0%)
      - initial_early_stage_rat

,firmname,year,firmage,firm_hq


#### Initial Network Partner 정보 missing 원인 파악

In [ ]:
# Centrality NA handling per config
from vc_analysis.config import parameters as params_mod

# Load parameters
params = params_mod.DEFAULT_PARAMS
cent_params = params.centrality

# Create in_network dummy: 1 if any centrality is present, else 0
centrality_cols = ['dgr_cent', 'btw_cent', 'pwr_max', 'pwr_p0', 'pwr_p75', 'pwr_p99', 'constraint', 'ego_dens']
existing_cent_cols = [c for c in centrality_cols if c in final_df.columns]

if cent_params.create_in_network_dummy and existing_cent_cols:
    final_df['in_network'] = (~final_df[existing_cent_cols].isna()).any(axis=1).astype(int)

# Optionally zero-fill selected measures when missing
if cent_params.fill_missing_centrality_as_zero and cent_params.zero_fill_columns:
    for col in cent_params.zero_fill_columns:
        if col in final_df.columns:
            final_df[col] = final_df[col].fillna(0)

print("✅ Centrality NA handling applied")
if 'in_network' in final_df.columns:
    print("   - in_network dummy added:", final_df['in_network'].value_counts().to_dict())
if cent_params.fill_missing_centrality_as_zero and cent_params.zero_fill_columns:
    print("   - Zero-filled columns:", [c for c in cent_params.zero_fill_columns if c in final_df.columns])


✅ Centrality NA handling applied
   - in_network dummy added: {1: 7234, 0: 2579}


In [ ]:
# === Initial status NaN 원인 플래그 (압축/안전 버전) ===

# 1) initial_* 집계 컬럼
initial_status_cols = [c for c in final_df.columns if c.startswith('initial_') and c.endswith(('_mean', '_max', '_min'))]

# 2) full history 기반 최초연도 맵 생성
firm_initial_full = (
    full_initial_year_df[['firmname', 'initial_year']].drop_duplicates('firmname')
    if 'full_initial_year_df' in globals() and not full_initial_year_df.empty
    else data['round'][['firmname', 'year']].dropna().groupby('firmname', as_index=False)['year'].min().rename(columns={'year':'initial_year'})
)
firm_initial_full = firm_initial_full.rename(columns={'initial_year':'initial_year_full'})

# 3) final_df에 병합(임시 프레임)
tmp = final_df.merge(firm_initial_full, on='firmname', how='left')

# 4) 보조 시리즈/마스크 (안전 접근)
iyf = tmp['initial_year_full'] if 'initial_year_full' in tmp.columns else pd.Series(np.nan, index=tmp.index)
iy  = tmp['initial_year']      if 'initial_year'      in tmp.columns else pd.Series(np.nan, index=tmp.index)

initial_year_present = iy.notna()
partners_zero_or_na = (
    (tmp['n_initial_partners'].isna() if 'n_initial_partners' in tmp.columns else True) |
    (tmp.get('n_initial_partners', 0) == 0) |
    (tmp['n_partner_years'].isna() if 'n_partner_years' in tmp.columns else True) |
    (tmp.get('n_partner_years', 0) == 0)
)
initial_status_all_nan = (tmp[initial_status_cols].isna().all(axis=1)) if initial_status_cols else pd.Series(False, index=tmp.index)

# 5) 분류 플래그
outside_cohort = iyf.notna() & ~iyf.between(START_YEAR, END_YEAR) & iy.isna()
no_partners_at_foundation = initial_year_present & partners_zero_or_na
has_partners_but_no_centrality = initial_year_present & ~partners_zero_or_na & initial_status_all_nan
missing_other = initial_status_all_nan & ~outside_cohort & ~no_partners_at_foundation & ~has_partners_but_no_centrality

# 6) 플래그 적용
tmp['initial_status_missing'] = initial_status_all_nan.astype(int)
tmp['initial_missing_outside_cohort'] = outside_cohort.astype(int)
tmp['initial_missing_no_partners'] = no_partners_at_foundation.astype(int)
tmp['initial_missing_no_centrality'] = has_partners_but_no_centrality.astype(int)
tmp['initial_missing_other'] = missing_other.astype(int)

# 7) ‘other’ → ‘no_partners’ 재분류 (코호트 내 최초연도는 있으나 t1~t3 파트너가 전혀 없던 firm)
firms_with_partners = set(initial_ties_df['firmname']) if 'initial_ties_df' in globals() else set()
no_partners_strict = (
    tmp['initial_status_missing'].eq(1) &
    iyf.between(START_YEAR, END_YEAR) &
    ~tmp['firmname'].isin(firms_with_partners)
)
tmp['initial_missing_no_partners'] = (tmp['initial_missing_no_partners'].astype(bool) | no_partners_strict).astype(int)
tmp['initial_missing_other'] = (
    tmp['initial_status_missing'].eq(1) &
    tmp['initial_missing_outside_cohort'].eq(0) &
    tmp['initial_missing_no_partners'].eq(0) &
    tmp['initial_missing_no_centrality'].eq(0)
).astype(int)

# 8) 최종 반영 + 요약
final_df = tmp
print("🔎 Initial status NaN 원인 플래그 요약:")
for col in ['initial_status_missing','initial_missing_outside_cohort','initial_missing_no_partners','initial_missing_no_centrality','initial_missing_other']:
    cnt = int(final_df[col].sum())
    print(f"  - {col}: {cnt:,} ({cnt/len(final_df)*100:.1f}%)")

🔎 Initial status NaN 원인 플래그 요약:
  - initial_status_missing: 5,433 (55.4%)
  - initial_missing_outside_cohort: 4,371 (44.5%)
  - initial_missing_no_partners: 913 (9.3%)
  - initial_missing_no_centrality: 0 (0.0%)
  - initial_missing_other: 168 (1.7%)


#### Final Sampling

In [ ]:
# === 분석 가능 샘플 필터링 (최종 분석용) ===
# 설정: 필요한 조건만 True로 켜서 사용
ANALYSIS_REQUIREMENTS = {
    'require_year_range': True,         # START_YEAR~END_YEAR만
    'require_firm_basics': True,        # firmage, firm_hq 결측 없음
    'require_centrality': True,         # 어떤 중심성이라도 관측(in_network=1 또는 중앙성 컬럼 중 1개 이상 notna)
    'require_initial_status': False,    # initial_year 존재 (initial_* 사용 분석 시 True)
    'require_performance': False        # perf_* 결측 없음(보통 0-치환되어 True)
}

df0 = final_df

# 1) 조건별 마스크 구성
year_ok = df0['year'].between(START_YEAR, END_YEAR) if ANALYSIS_REQUIREMENTS['require_year_range'] else pd.Series(True, index=df0.index)

basics_cols = [c for c in ['firmage','firm_hq'] if c in df0.columns]
basics_ok = df0[basics_cols].notna().all(axis=1) if (ANALYSIS_REQUIREMENTS['require_firm_basics'] and basics_cols) else pd.Series(True, index=df0.index)

cent_cols = [c for c in ['dgr_cent','btw_cent','pwr_max','pwr_p0','pwr_p75','pwr_p99','constraint','ego_dens'] if c in df0.columns]
if ANALYSIS_REQUIREMENTS['require_centrality']:
    if 'in_network' in df0.columns:
        centrality_ok = df0['in_network'].eq(1)
    else:
        centrality_ok = df0[cent_cols].notna().any(axis=1) if cent_cols else pd.Series(False, index=df0.index)
else:
    centrality_ok = pd.Series(True, index=df0.index)

initial_ok = df0['initial_year'].notna() if ANALYSIS_REQUIREMENTS['require_initial_status'] else pd.Series(True, index=df0.index)

perf_cols = [c for c in ['perf_IPO','perf_MnA','perf_all'] if c in df0.columns]
performance_ok = df0[perf_cols].notna().all(axis=1) if (ANALYSIS_REQUIREMENTS['require_performance'] and perf_cols) else pd.Series(True, index=df0.index)

# 2) 종합 마스크 및 필터링
eligible_mask = year_ok & basics_ok & centrality_ok & initial_ok & performance_ok
analysis_df = df0.loc[eligible_mask].copy()

# 3) 요약
print("✅ 분석 가능 샘플 필터링 완료")
print(f"  - 전체: {len(df0):,}  →  분석 가능: {len(analysis_df):,}  (잔존 {len(analysis_df)/len(df0)*100:.1f}%)")
drop_reasons = {
    'year_range': (~year_ok).sum(),
    'firm_basics': (~basics_ok).sum(),
    'centrality': (~centrality_ok).sum(),
    'initial_status': (~initial_ok).sum(),
    'performance': (~performance_ok).sum()
}
print("  - Drop counts (조건별 중복 집계):", {k: int(v) for k, v in drop_reasons.items()})

# 필요 시 최종 데이터 교체
# final_df = analysis_df

✅ 분석 가능 샘플 필터링 완료
  - 전체: 9,813  →  분석 가능: 7,234  (잔존 73.7%)
  - Drop counts (조건별 중복 집계): {'year_range': 0, 'firm_basics': 0, 'centrality': 2579, 'initial_status': 0, 'performance': 0}


In [ ]:
# === 최종 분석 데이터 저장 (R에서 바로 사용 가능) ===
from pathlib import Path
from datetime import datetime

# 0) 설정
CSV_SAMPLE_N = 1000          # CSV로 저장할 샘플 수 (None 또는 0이면 CSV 저장 생략)
CSV_SAMPLE_RANDOM = True     # True: 무작위 샘플, False: 상위 N개
CSV_SAMPLE_SEED = 42         # 무작위 샘플 시드

OUTPUT_DIR = Path("./analysis_outputs")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# 날짜시간 스탬프 (예: 251028_2359)
STAMP = datetime.now().strftime("%y%m%d_%H%M")

# 1) 인덱스 리셋
analysis_df = analysis_df.reset_index(drop=True)

# 2) 파일 경로 (스탬프 추가)
parquet_path = OUTPUT_DIR / f"final_analysis_{START_YEAR}_{END_YEAR}_{STAMP}.parquet"
feather_path = OUTPUT_DIR / f"final_analysis_{START_YEAR}_{END_YEAR}_{STAMP}.feather"
csv_path = OUTPUT_DIR / f"final_analysis_sample_{START_YEAR}_{END_YEAR}_n{CSV_SAMPLE_N}_{STAMP}.csv"

# 3) Parquet 저장 (R: arrow::read_parquet)
analysis_df.to_parquet(parquet_path, index=False)

# 4) Feather 저장 (R: arrow::read_feather)
try:
    analysis_df.to_feather(feather_path)
except Exception as e:
    print(f"Feather 저장 생략(에러): {e}")

# 5) CSV 샘플 저장 (용량 제한용)
if CSV_SAMPLE_N and CSV_SAMPLE_N > 0:
    if CSV_SAMPLE_N >= len(analysis_df):
        csv_sample = analysis_df
    else:
        csv_sample = (
            analysis_df.sample(n=CSV_SAMPLE_N, random_state=CSV_SAMPLE_SEED)
            if CSV_SAMPLE_RANDOM else
            analysis_df.head(CSV_SAMPLE_N)
        )
    csv_sample.to_csv(csv_path, index=False)

print("✅ 저장 완료")
print(f"  - Parquet: {parquet_path}")
print(f"  - Feather: {feather_path}")
if CSV_SAMPLE_N and CSV_SAMPLE_N > 0:
    print(f"  - CSV(sample): {csv_path} ({len(csv_sample):,} rows)")

✅ 저장 완료
  - Parquet: analysis_outputs/final_analysis_1990_2000_251110_0033.parquet
  - Feather: analysis_outputs/final_analysis_1990_2000_251110_0033.feather
  - CSV(sample): analysis_outputs/final_analysis_sample_1990_2000_n1000_251110_0033.csv (1,000 rows)


In [ ]:
"""
================================================================================
진단: Initial Ties 매칭 문제 확인 (해결됨)
================================================================================

진단 결과:
----------
1. 데이터 크기:
   - firm_vars_df_filtered: 10,431 rows, 3,140 firms (1990~2000년 투자 집행)
   - initial_ties_df: 1,720 rows, 1,720 firms (1990~2000년 initial ties 형성)

2. 공통 Firm:
   - 공통 firm: 1,720개 ✅
   - firm_vars only: 1,420개 (initial ties 없음)
   - initial_ties only: 0개 (모두 매칭됨)

3. Missing 58.2%의 의미:
   - 3,140개 firm 중 1,720개만 1990~2000년에 initial ties를 형성
   - 나머지 1,420개 firm은:
     * 1990년 이전에 이미 initial ties를 형성했거나 (established firms)
     * 데이터에 initial ties 정보가 없는 경우
   
4. 결론:
   - Merge는 정상적으로 작동하고 있습니다! ✅
   - Missing은 의도된 결과입니다 (1990년 이전 established firms)
   - R 코드도 동일한 로직입니다
   
5. 분석 시 해석:
   - initial_year가 있는 1,720 firms: 1990~2000 코호트 (treatment group)
   - initial_year가 NaN인 1,420 firms: 통제 그룹 (established firms)

================================================================================
"""

# # 진단 코드 (필요 시 주석 해제하여 실행)
# print("=" * 80)
# print("진단: Initial Ties 매칭 문제")
# print("=" * 80)

# # 1. 데이터셋 기본 정보
# print(f"\n1. 데이터셋 크기:")
# print(f"   - firm_vars_df_filtered: {len(firm_vars_df_filtered)} rows, {firm_vars_df_filtered['firmname'].nunique()} firms")
# print(f"   - initial_ties_df: {len(initial_ties_df)} rows, {initial_ties_df['firmname'].nunique()} firms")

# # 2. Firmname 샘플 비교
# print(f"\n2. Firmname 샘플 비교:")
# print(f"\n   firm_vars_df_filtered (상위 5개):")
# print(firm_vars_df_filtered['firmname'].head().tolist())
# print(f"\n   initial_ties_df (상위 5개):")
# print(initial_ties_df['firmname'].head().tolist())

# # 3. 공통 firm 확인
# common_firms = set(firm_vars_df_filtered['firmname']) & set(initial_ties_df['firmname'])
# print(f"\n3. 공통 Firm:")
# print(f"   - 공통 firm 수: {len(common_firms)}")
# print(f"   - firm_vars only: {firm_vars_df_filtered['firmname'].nunique() - len(common_firms)}")
# print(f"   - initial_ties only: {initial_ties_df['firmname'].nunique() - len(common_firms)}")

# # 4. 샘플 firm 추적
# if len(common_firms) > 0:
#     sample_firm = list(common_firms)[0]
#     print(f"\n4. 샘플 Firm 추적: {sample_firm}")
#     print(f"\n   firm_vars_df_filtered:")
#     print(firm_vars_df_filtered[firm_vars_df_filtered['firmname'] == sample_firm][['firmname', 'year']].head())
#     print(f"\n   initial_ties_df:")
#     print(initial_ties_df[initial_ties_df['firmname'] == sample_firm][['firmname', 'initial_year']].head())
# else:
#     print(f"\n4. ⚠️  공통 firm이 없습니다! Firmname 불일치 문제!")
    
# # 5. Merge 테스트
# print(f"\n5. Merge 테스트:")
# test_merge = firm_vars_df_filtered[['firmname', 'year']].head(10).merge(
#     initial_ties_df[['firmname', 'initial_year']],
#     on='firmname',
#     how='left'
# )
# print(test_merge)


'\n================================================================================\n진단: Initial Ties 매칭 문제 확인 (해결됨)\n================================================================================\n\n진단 결과:\n----------\n1. 데이터 크기:\n   - firm_vars_df_filtered: 10,431 rows, 3,140 firms (1990~2000년 투자 집행)\n   - initial_ties_df: 1,720 rows, 1,720 firms (1990~2000년 initial ties 형성)\n\n2. 공통 Firm:\n   - 공통 firm: 1,720개 ✅\n   - firm_vars only: 1,420개 (initial ties 없음)\n   - initial_ties only: 0개 (모두 매칭됨)\n\n3. Missing 58.2%의 의미:\n   - 3,140개 firm 중 1,720개만 1990~2000년에 initial ties를 형성\n   - 나머지 1,420개 firm은:\n     * 1990년 이전에 이미 initial ties를 형성했거나 (established firms)\n     * 데이터에 initial ties 정보가 없는 경우\n\n4. 결론:\n   - Merge는 정상적으로 작동하고 있습니다! ✅\n   - Missing은 의도된 결과입니다 (1990년 이전 established firms)\n   - R 코드도 동일한 로직입니다\n\n5. 분석 시 해석:\n   - initial_year가 있는 1,720 firms: 1990~2000 코호트 (treatment group)\n   - initial_year가 NaN인 1,420 firms: 통제 그룹 (established firms)\n\n========================

In [ ]:
# # 메모리 정리: 중간 변수 제거
# import gc

# # 제거할 변수 (네트워크 객체 + 중간 데이터)
# variables_to_delete = [
#     'networks', 'imprinting_networks',  # 네트워크 (가장 큰 메모리)
#     'imprinting_centrality_df', 'centrality_df_filtered',  # 중간 중심성
#     'filtered_round', 'initial_partners_df', 'initial_ties_with_cent',  # 중간 처리
#     'full_initial_year_df', 'initial_year_df',  # 중간 연도
#     'firm_vars_df', 'initial_ties_df_unique'  # 필터링 전
# ]

# # 제거 실행
# for var_name in variables_to_delete:
#     if var_name in globals():
#         del globals()[var_name]

# gc.collect()

# print("✅ 메모리 정리 완료")
# print(f"   제거: {len(variables_to_delete)}개 변수")
# print(f"\n유지 데이터:")
# print(f"   - data: 원본 데이터")
# print(f"   - centrality_df: 전체 중심성")
# print(f"   - firm_vars_df_filtered: Firm 변수")
# print(f"   - initial_ties_df: Initial Partner Status")
# print(f"   - final_df: 최종 분석 데이터 ⭐")

